In [56]:

import numpy as np
import pandas as pd

import pickle

intermediate_data = pickle.load(open("../data/nfl-big-data-bowl-2024/_play_by_play_intermediate.pkl", "rb"))


In [57]:
test_play = intermediate_data.iloc[0]
print(test_play.keys())
tracking_data = test_play["player_tracking"]
ball_carrier_id = test_play["play_features"].loc["ballCarrierId"]
ball_carrier_tracking_data = tracking_data.loc[:, ball_carrier_id]


dict_keys(['game_id', 'play_id', 'player_tracking', 'event_timeseries', 'players_on_the_field', 'play_features', 'tackle_successful', 'yards_after_contact'])


In [75]:
def get_relative_features_to_ball_carrier(group):
    group_ = group.T.droplevel(0, axis=1)
    dist = np.sqrt(np.square(group_.loc[:, ['x', 'y']] - ball_carrier_tracking_data.loc[:, ['x', 'y']]).sum(axis=1))
    
    delta_angle = ball_carrier_tracking_data.loc[:, 'dir'] - group_.loc[:, 'dir']

    player_speed = group_.loc[:, 's']
    ball_carrier_speed = ball_carrier_tracking_data.loc[:, 's']
    rel_spd = np.sqrt(player_speed ** 2 + ball_carrier_speed ** 2 - 2 * player_speed * ball_carrier_speed * np.cos(delta_angle))

    player_accel = group_.loc[:, 'a']
    ball_carrier_accel = ball_carrier_tracking_data.loc[:, 'a']
    rel_accel = np.sqrt(player_accel ** 2 + ball_carrier_accel ** 2 - 2 * player_accel * ball_carrier_accel * np.cos(delta_angle))
    
    return pd.DataFrame({
        "distance": dist,
        "relative_speed": rel_spd,
        "relative_acceleration": rel_accel,
        "delta_angle": delta_angle,
        "cosine_similarity": np.cos(delta_angle),
    })

non_ball_carriers = tracking_data.T.groupby(level=0).filter(lambda g: g.name != ball_carrier_id)
geometric_features = non_ball_carriers.groupby(level=0).apply(get_relative_features_to_ball_carrier)
geometric_features


distance  relative_speed  relative_acceleration  delta_angle  \
35472.0 0   4.240814        6.255264               3.332859       153.53   
        1   4.471532        6.467044               4.004275       160.74   
        2   4.800021        6.663768               4.336356       166.06   
        3   5.204652        6.795110               4.853028       172.40   
        4   5.678424        6.713280               5.328899       179.56   
...              ...             ...                    ...          ...   
53532.0 15  2.951491       11.578340               2.817839       317.90   
        16  2.432879        4.241660               1.507858       320.87   
        17  1.952562       12.350827               3.014634       -34.26   
        18  1.550290       12.325017               4.070641       -27.94   
        19  1.188276        6.765808               2.428443       -24.06   

            cosine_similarity  
35472.0 0           -0.917900  
        1           -0.868427  
        2           -0.902864  
        3           -0.925820  
        4           -0.882700  
...                       ...  
53532.0 15          -0.825820  
        16           0.909985  
        17          -0.956067  
        18          -0.944629  
        19           0.477718  

[420 rows x 5 columns]

In [88]:
onfield = test_play['players_on_the_field']
play_features = test_play['play_features']
offense = onfield.loc[(onfield["club"] == play_features["possessionTeam"]) & (onfield["nflId"] != play_features["ballCarrierId"]), "nflId"]
defense = onfield.loc[onfield["club"] == play_features["defensiveTeam"], "nflId"]

offense_feature_groups = geometric_features.loc[pd.IndexSlice[offense]].groupby(level=0)
defense_feature_groups = geometric_features.loc[pd.IndexSlice[defense]].groupby(level=0)

offense_features = pd.concat([g.reset_index(drop=True) for _, g in offense_feature_groups], axis=1, keys=offense_feature_groups.groups.keys())
defense_features =  pd.concat([g.reset_index(drop=True) for _, g in defense_feature_groups], axis=1, keys=defense_feature_groups.groups.keys())
offense_raw = tracking_data.loc[:, pd.IndexSlice[offense]]
defense_raw = tracking_data.loc[:, pd.IndexSlice[defense]]


In [90]:
offense_raw


35472.0                                    42392.0                     ...  \
         x      y     s     a       o     dir       x      y     s     a  ...   
0    82.42  26.13  2.43  0.37  196.42  140.67   81.75  28.39  1.18  2.36  ...   
1    82.58  25.96  2.34  0.93  203.85  138.66   81.71  28.53  1.50  2.17  ...   
2    82.73  25.79  2.20  1.39  209.50  136.40   81.65  28.70  1.80  1.69  ...   
3    82.87  25.66  1.96  1.87  214.57  133.71   81.60  28.92  2.15  1.52  ...   
4    82.99  25.55  1.62  2.42  222.35  130.47   81.53  29.16  2.45  1.33  ...   
5    83.09  25.48  1.19  2.80  234.72  125.05   81.46  29.44  2.75  1.08  ...   
6    83.16  25.45  0.83  2.89  246.19  114.56   81.37  29.76  3.08  0.95  ...   
7    83.21  25.45  0.47  2.88  265.22   91.69   81.28  30.10  3.36  0.75  ...   
8    83.23  25.47  0.32  2.69  280.09   35.37   81.17  30.44  3.53  0.58  ...   
9    83.23  25.51  0.51  2.50  284.57  354.43   81.06  30.81  3.75  0.80  ...   
10   83.20  25.57  0.71  1.99  294.98  338.26   80.92  31.18  3.97  1.09  ...   
11   83.14  25.66  0.98  1.68  303.96  327.69   80.77  31.57  4.13  0.86  ...   
12   83.07  25.75  1.18  1.27  306.53  322.46   80.59  31.96  4.21  1.05  ...   
13   82.97  25.86  1.37  0.94  307.91  318.58   80.38  32.37  4.36  1.33  ...   
14   82.87  25.96  1.48  0.60  313.14  316.52   80.16  32.77  4.42  1.49  ...   
15   82.76  26.08  1.57  0.47  320.43  315.12   79.91  33.18  4.51  1.57  ...   
16   82.63  26.20  1.66  0.34  321.54  314.17   79.64  33.60  4.56  1.64  ...   
17   82.51  26.33  1.76  0.10  319.48  314.90   79.36  34.01  4.62  1.60  ...   
18   82.38  26.46  1.77  0.24  323.26  314.81   79.07  34.39  4.65  1.33  ...   
19   82.24  26.60  1.84  0.32  324.35  315.14   78.77  34.77  4.65  1.46  ...   

   52536.0                       53522.0                                     
         s     a       o     dir       x      y     s     a       o     dir  
0     3.43  3.38   19.41  311.87   86.28  33.25  1.57  3.75   20.01  118.94  
1     3.22  2.99   15.54  315.91   86.38  33.18  1.10  3.75   11.69  128.49  
2     3.07  2.54   12.06  319.75   86.44  33.12  0.72  3.68  359.65  146.54  
3     2.92  2.30    9.05  323.94   86.44  33.06  0.53  3.51  347.62  189.50  
4     2.83  1.98    6.72  327.64   86.41  33.02  0.66  3.04  336.17  230.74  
5     2.75  1.88    3.59  331.84   86.34  32.99  0.89  2.35  326.75  248.85  
6     2.67  1.56  359.13  334.34   86.22  32.96  1.17  1.97  312.54  259.14  
7     2.64  1.15  356.36  335.96   86.07  32.95  1.46  1.61  312.54  265.54  
8     2.65  0.77  354.64  337.12   85.89  32.95  1.75  1.40  307.71  271.29  
9     2.69  0.33  351.33  336.95   85.69  32.98  1.98  1.17  303.63  275.79  
10    2.72  0.20  348.99  337.74   85.47  33.03  2.17  1.01  297.70  280.71  
11    2.77  0.19  350.23  338.73   85.24  33.10  2.33  1.07  294.91  285.13  
12    2.80  0.37  349.54  340.68   84.99  33.18  2.54  1.25  295.83  289.07  
13    2.88  0.36  349.54  341.30   84.73  33.29  2.71  1.12  295.83  292.52  
14    2.93  0.51  350.35  343.01   84.44  33.42  2.94  0.99  295.83  294.96  
15    3.04  0.48  351.04  343.08   84.17  33.57  3.06  0.90  299.23  297.51  
16    3.12  0.39  351.63  343.20   83.86  33.74  3.30  0.88  302.88  299.79  
17    3.18  0.43  353.85  344.96   83.56  33.93  3.43  0.79  301.48  301.97  
18    3.28  0.45  354.86  345.93   83.24  34.14  3.61  0.73  298.17  303.91  
19    3.36  0.34  357.18  345.99   82.94  34.35  3.67  0.76  298.92  305.09  

[20 rows x 60 columns]

In [82]:
geometric_features.loc[pd.IndexSlice[[46076], :]]


KeyError: '[46076] not in index'